In [1]:
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import pandas as pd
import numpy as np
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2025-01-25 13:37:46.978752: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-25 13:37:47.130738: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737812267.188130    5482 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737812267.204785    5482 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-25 13:37:47.344308: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Num GPUs Available:  1


In [2]:
df = pd.read_csv('../data/cereal.csv')
df.head()

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843


In [19]:
df.cups.min()

np.float64(0.25)

In [21]:
df['potass'] = df['potass'].replace(-1, np.median(df['potass']))
df['carbo'] = df['carbo'].replace(-1, np.median(df['carbo']))
df['sugars'] = df['sugars'].replace(-1, np.median(df['sugars']))

In [41]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

# Define the specific columns for linear regression
numeric_features = ['calories', 'protein', 'fat', 'sodium', 'fiber', 'carbo', 'sugars', 'potass', 'vitamins']
selected_numeric_features = numeric_features
selected_categorical_features = []

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), selected_numeric_features),
        ('cat', OneHotEncoder(), selected_categorical_features)
    ])

# Split the data
X = df.drop(['name', 'rating'], axis=1)
y = df['rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
preprocessor.fit(X_train)
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [25]:
# from sklearn.linear_model import LinearRegression
# lr = LinearRegression()
# lr.fit(X_train_processed, y_train)
# print(f"Linear Regression Test R²: {lr.score(X_test_processed, y_test):.2%}")

Linear Regression Test R²: 99.61%


In [43]:
lr = LinearRegression()
lr.fit(X_train_processed, y_train)

# Evaluate the model
print(f"Linear Regression Test R²: {lr.score(X_test_processed, y_test):.2%}")

Linear Regression Test R²: 99.49%
